In [17]:
import elevenlabs

import os
import base64
import pygame
import time

In [18]:
generated_text=input("Enter a text")

In [19]:
from elevenlabs import ElevenLabs

client=ElevenLabs(api_key=os.getenv('ELEVENLABS_API_KEY'))


response=client.text_to_speech.convert_with_timestamps(
    voice_id="NFG5qt843uXKj4pFvR7C",
    text=generated_text
)

In [21]:
audio=base64.b64decode(response.audio_base_64)

with open('prime.mp3','wb') as file:
    file.write(audio)

In [22]:
alignment = response.alignment
chars = alignment.characters
starts = alignment.character_start_times_seconds
ends = alignment.character_end_times_seconds

In [23]:
def func(response):
    alignment = response.alignment
    chars = alignment.characters
    starts = alignment.character_start_times_seconds
    ends = alignment.character_end_times_seconds
    words = []
    current_word = ""
    current_start = None

    for c, s, e in zip(chars, starts, ends):
        if c != " ":
            if current_word == "":  # starting a new word
                current_start = s
            current_word += c
            current_end = e
        else:
            if current_word:  # end of a word
                words.append((current_word, current_start, current_end))
                current_word = ""

    # Add the last word if there is one
    if current_word:
        words.append((current_word, current_start, current_end))

    # Print result
    for word, s, e in words:
        print(f"'{word}' spoken from {s:.2f}s to {e:.2f}s")


    return words


In [32]:
def group_words_into_phrases(words, max_words_per_phrase=3):
    """Groups individual words into phrases based on punctuation or max word count."""
    phrases = []
    current_phrase = []
    if not words:
        return []

    phrase_start_time = words[0][1]

    for i, (word, start, end) in enumerate(words):
        current_phrase.append(word)
        should_end_phrase = any(c in word for c in ",.?!") or (len(current_phrase) >= max_words_per_phrase or i == len(words) - 1)

        if should_end_phrase:
            phrase_text = " ".join(current_phrase)
            phrases.append((phrase_text, phrase_start_time, end))
            current_phrase = []
            if i < len(words) - 1:
                phrase_start_time = words[i+1][1]

    return phrases

In [33]:
timestamp=func(response)

'Skibidi' spoken from 0.00s to 0.61s
'bop,' spoken from 0.67s to 1.14s
'yes' spoken from 1.30s to 1.60s
'yes' spoken from 1.71s to 2.01s
'yes!' spoken from 2.11s to 2.67s
'I' spoken from 2.93s to 2.98s
'woke' spoken from 3.09s to 3.35s
'up' spoken from 3.39s to 3.52s
'on' spoken from 3.58s to 3.66s
'a' spoken from 3.70s to 3.73s
'rainbow' spoken from 3.79s to 4.18s
'trampoline' spoken from 4.24s to 4.98s
'made' spoken from 5.12s to 5.38s
'of' spoken from 5.41s to 5.47s
'pickles.' spoken from 5.53s to 6.26s
'My' spoken from 6.99s to 7.12s
'alarm' spoken from 7.17s to 7.45s
'clock' spoken from 7.51s to 7.78s
'was' spoken from 7.85s to 7.98s
'singing' spoken from 8.02s to 8.37s
'jazz' spoken from 8.47s to 8.88s
'in' spoken from 8.99s to 9.11s
'Morse' spoken from 9.17s to 9.47s
'code.' spoken from 9.53s to 10.05s
'Tung-tung!' spoken from 10.22s to 10.95s
'My' spoken from 11.24s to 11.37s
'socks' spoken from 11.45s to 11.81s
'decided' spoken from 11.87s to 12.46s
'they' spoken from 12.52s t

In [34]:
phrases=group_words_into_phrases(timestamp)

In [35]:
phrases

[('Skibidi bop,', 0.0, 1.138),
 ('yes yes yes!', 1.3, 2.67),
 ('I woke up', 2.926, 3.518),
 ('on a rainbow', 3.576, 4.18),
 ('trampoline made of', 4.238, 5.468),
 ('pickles.', 5.526, 6.258),
 ('My alarm clock', 6.989, 7.779),
 ('was singing jazz', 7.848, 8.882),
 ('in Morse code.', 8.986, 10.054),
 ('Tung-tung!', 10.217, 10.948),
 ('My socks decided', 11.238, 12.458),
 ('they were sentient', 12.516, 13.468),
 ('and started a', 13.63, 14.385),
 ('revolution,', 14.454, 15.209),
 ('marching across the', 15.43, 16.393),
 ('kitchen while flinging', 16.451, 17.775),
 ('pancakes at the', 17.857, 18.634),
 ('ceiling.', 18.681, 19.203),
 ('The fridge opened', 19.83, 20.596),
 ('itself and whispered', 20.643, 21.479),
 ('secrets about the', 21.537, 22.326),
 ('moon’s secret twin,', 22.384, 23.673),
 ('which apparently is', 23.778, 24.648),
 ('a giant disco', 24.683, 25.682),
 ('pineapple.', 25.751, 26.471),
 ('I tried to', 27.098, 27.586),
 ('leave,', 27.655, 28.038),
 ('but my shoes', 28.143, 2

In [28]:
pygame.mixer.init()
pygame.mixer.music.load("prime.mp3")

In [29]:
import ffmpeg

def add_timed_text(stream, text, start, end, fontsize_ratio=0.12, fontcolor="white", outline=6):
    """
    Adds large, bold, centered YouTube Shorts-style text.
    """
    return stream.drawtext(
        text=text,
        fontfile=r"C:\Windows\Fonts\impact.ttf",   # Bold meme/viral look
        fontsize=f"h*{fontsize_ratio}",            # Scale with video height
        fontcolor=fontcolor,
        x="(w-text_w)/2",                          # Center horizontally
        y="(h-text_h)/2",                          # Center vertically
        borderw=outline,                           # Thick outline
        bordercolor="black",
        box=0,                                     # No box, outline only
        enable=f"between(t,{start},{end})"
    )


In [30]:
def overlay(stream,timestamp):
   '''
   Function: Apply timestamp

   Args:
      stream: Canvas to apply work
      timestamp: Words with start and end time
   '''
   for word,start,end in timestamp:
       stream=add_timed_text(stream,word,start,end)
    
   return stream
    


In [31]:
import ffmpeg

input_file = "video.mp4"
audio_file = "prime.mp3"
output_file = "asscream.mp4"  #mp4 to apply subtitles to

# Load video
video = ffmpeg.input(input_file).video
audio = ffmpeg.input(audio_file).audio

video=overlay(video,phrases)

(
    ffmpeg
    .output(video, audio, output_file, vcodec='libx264', acodec='aac', strict='experimental')
    .overwrite_output()
    .run()
)


(None, None)